In [1]:
import pandas as pd
import re
import numpy as np
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, chi2
import pickle
import random

In [20]:
p=0.4
df = pd.read_csv('datasets/cleaned_articles1.csv',header=None,skiprows=lambda i: 1>0 and random.random() > p)

df2 = pd.read_csv('datasets/cleaned_articles2.csv',header=None,skiprows=lambda i: 1>0 and random.random() > p)


In [21]:
df.head(5)

,0,1,2,3,4,5,6,7,8,9,10,11
0,NaN,Unnamed: 0,id,title,publication,author,date,year,month,url,content,cleaned
1,5.0,5,17288,"Sick With a Cold, Queen Elizabeth Misses New Y...",New York Times,Sewell Chan,2017-01-02,2017.0,1.0,NaN,"LONDON — Queen Elizabeth II, who has been b...",london queen elizabeth ii battl cold week miss...
2,7.0,7,17290,"After ‘The Biggest Loser,’ Their Bodies Fought...",New York Times,Gina Kolata,2017-02-08,2017.0,2.0,NaN,"Danny Cahill stood, slightly dazed, in a blizz...",danni cahil stood slight daze blizzard confett...
3,11.0,11,17294,Can Carbon Capture Technology Prosper Under Tr...,New York Times,John Schwartz,2017-01-05,2017.0,1.0,NaN,"THOMPSONS, Tex. — Can one of the most promi...",thompson tex can one promis troubl technolog f...
4,16.0,16,17300,Dylann Roof Himself Rejects Best Defense Again...,New York Times,Kevin Sack and Alan Blinder,2017-01-02,2017.0,1.0,NaN,pages into the journal found in Dylann S. ...,page journal found dylann s roof car assert bl...


In [6]:
stemmer = SnowballStemmer('english')
words = stopwords.words("english")

In [31]:
df.columns = ['NaN', 'Unnamed', 'id', 'title', 'publication', 'author', 'date', 'year', 'month', 'url', 'content', 'ceaned']

In [32]:
df[df.publication == 'Breitbart']

,NaN,Unnamed,id,title,publication,author,date,year,month,url,content,ceaned
3054,7803.0,7803,26539,CNN’s Zeleny: ’Hard to Imagine’ Obama Would Ha...,Breitbart,Ian Hanchett,2017-01-17,2017.0,1.0,NaN,On Tuesday’s broadcast of CNN’s “Situation Roo...,on tuesday broadcast cnn situat room cnn senio...
3055,7804.0,7804,26540,American Students on Spring Break Chant ’Build...,Breitbart,Katherine Rodriguez,2017-03-21,2017.0,3.0,NaN,A group of American spring break revelers repo...,a group american spring break revel report cha...
3056,7808.0,7808,26544,Watch: The Resistance Creates Anti-Trump Aerob...,Breitbart,Amanda House,2017-05-13,2017.0,5.0,NaN,Two California women have choreographed an a...,two california women choreograph aerob class r...
3057,7810.0,7810,26546,20-Year-Old Brit Dies Fighting Islamic State i...,Breitbart,Nick Hallett,2017-01-03,2017.0,1.0,NaN,A British man who traveled to Syria to figh...,a british man travel syria fight islam state k...
3058,7812.0,7812,26548,Drug Stocks Drop as Trump Backs Price Bidding,Breitbart,Dr. Susan Berry,2017-01-11,2017.0,1.0,NaN,Drug stocks plummeted as Donald Trump announ...,drug stock plummet donald trump announc want n...
...,...,...,...,...,...,...,...,...,...,...,...,...
12497,31567.0,31575,50341,Obama Plans to Bring Thousands More Syrian Ref...,Breitbart,Warner Todd Huston,2016-03-02,2016.0,3.0,NaN,President Barack Obama has been quietly pushin...,presid barack obama quiet push new plan bring ...
12498,31570.0,31578,50344,Zika Virus Found in Danish Tourist Returning F...,Breitbart,AFP,2016-01-27,2016.0,1.0,NaN,A Danish tourist returning from Latin America ...,a danish tourist return latin america diagnos ...
12499,31572.0,31580,50346,FLASHBACK: How the Clinton Campaign Demonized ...,Breitbart,Ben Kew,2016-10-28,2016.0,10.0,NaN,Hillary Clinton’s campaign chief John Podesta ...,hillari clinton campaign chief john podesta ca...
12500,31578.0,31586,50352,IDF Deploys Reinforcements To West Bank Follow...,Breitbart,Deborah Danan,2016-09-18,2016.0,9.0,NaN,TEL AVIV — Following six terror attacks in ...,tel aviv follow six terror attack two day jeru...


In [22]:
#Second batch:
n_s_breitbart = df[df['4'] == 'Breitbart']
n_s_times = df[df['4'] == 'New York Times']
#First batch:
n_s_atlantic = df2[df2['4'] == 'Atlantic']
n_s_post = df2[df2['4'] == 'New York Post']

KeyError: '4'

,Unnamed: 0,Unnamed: 0.1,id,title,publication,author,date,year,month,url,content,cleaned
27633,27633,81092,122293,Gunman dressed as Santa Claus kills dozens at ...,New York Post,Associated Press,2016-12-31,2016.0,12.0,https://web.archive.org/web/20170101013549/htt...,ISTANBUL — An assailant believed to have be...,istanbul an assail believ dress santa claus co...
27634,27634,81093,122294,Disney could receive $50M for Carrie Fisher’s ...,New York Post,Lindsay Putnam,2016-12-31,2016.0,12.0,https://web.archive.org/web/20170101013549/htt...,Carrie Fisher’s death could be a windfall for ...,carri fisher death could windfal disney the en...
27635,27635,81094,122295,Alabama torments Washington with old-school fo...,New York Post,Howie Kussoy,2016-12-31,2016.0,12.0,https://web.archive.org/web/20170101013549/htt...,ATLANTA — Washington wanted to prove the wo...,atlanta washington want prove world wrong earn...
27636,27636,81095,122296,"NBC, Charter extend talks to avert midnight bl...",New York Post,Post Staff Report,2016-12-31,2016.0,12.0,https://web.archive.org/web/20170101013549/htt...,A midnight blackout of NBC on local cable scre...,a midnight blackout nbc local cabl screen aver...
27637,27637,81096,122297,"Phil Jackson bolts, Greg Bird explodes and mor...",New York Post,Post Staff Report,2016-12-31,2016.0,12.0,https://web.archive.org/web/20170101013549/htt...,This is the best day of the year to be a sport...,this best day year sport fan well unless jet f...
...,...,...,...,...,...,...,...,...,...,...,...,...
45121,45121,98580,145561,Fresh scandal for Mayor de Blasio — and bad ne...,New York Post,Post Editorial Board,2016-05-05,2016.0,5.0,http://nypost.com/2016/05/05/fresh-scandal-for...,If Mayor Bill de Blasio personally sought dona...,if mayor bill de blasio person sought donat fo...
45122,45122,98581,145562,Bernie Sanders’ drive to keep people poor,New York Post,Jonah Goldberg,2016-04-08,2016.0,4.0,http://nypost.com/2016/04/08/bernie-sanders-dr...,"For years, supporters of free trade have been ...",for year support free trade tri reach bipartis...
45123,45123,98582,145563,Univision buys bankrupt Gawker for $135M,New York Post,Claire Atkinson and Julia Marsh,2016-08-16,2016.0,8.0,http://nypost.com/2016/08/16/univision-buys-ba...,Univision won the bidding war for Gawker Media...,univis bid war gawker media group tuesday stri...
45124,45124,98583,145564,Jury will decide fate of cop who fatally shot ...,New York Post,Lia Eustachewich,2016-01-06,2016.0,1.0,http://nypost.com/2016/01/06/jury-will-decide-...,A ​Brooklyn jury will decide the fate of an NY...,a brooklyn juri decid fate nypd offic charg co...


In [15]:
n_s = list(n_s_breitbart.iloc[:,11].values) + list(n_s_post.iloc[:,11].values) \
 + list(n_s_atlantic.iloc[:,11].values) + list(n_s_times.iloc[:,11].values)

In [16]:
n_s[0]

'on tuesday broadcast cnn situat room cnn senior washington correspond jeff zeleni state chelsea man transit man woman certain play presid obama decis commut man sentenc without hard imagin i think presid would done zeleni said i think question presid white hous answer answer i think import one much person stori chelsea man involv outcri left strong and difficult time feder prison question but central question without wonder outcom would i think might he ad b ecaus transit man woman i think certain play without hard imagin i think presid would done mediait follow ian hanchett twitter ianhanchett'

In [13]:
n_s = [word.replace('New York Post','') for word in n_s]
n_s = [word.replace('Breitbart','') for word in n_s]
n_s = [word.replace('New York Times','') for word in n_s]
n_s = [word.replace('Atlantic','') for word in n_s]

AttributeError: 'float' object has no attribute 'replace'

In [8]:
classes_Bias = np.asarray([1 for i in range(len(n_s_breitbart))] + \
[1 for i in range(len(n_s_post))] + [2 for i in range(len(n_s_atlantic))] + \
[2 for i in range(len(n_s_times))])

In [17]:
X_train, X_test, y_train, y_test = train_test_split(n_s, classes_Bias, test_size=0.2)

pipeline = Pipeline([('vect', TfidfVectorizer(ngram_range=(1, 2), stop_words="english", sublinear_tf=True)),
                     ('chi',  SelectKBest(chi2, k=10000)),
                     ('clf', LinearSVC(C=1.0, penalty='l1', max_iter=3000, dual=False))])

In [18]:
model = pipeline.fit(X_train, y_train)

vectorizer = model.named_steps['vect']
chi = model.named_steps['chi']
clf = model.named_steps['clf']

KeyboardInterrupt: 

In [ ]:
print("accuracy score: " + str(model.score(X_test, y_test)))

In [ ]:
n_s_atlantic.content[7]

In [34]:
print(model.predict(['They were primed to become eukaryotes. ” This debate is likely to rage on for a while, because no one has actually seen the Asgardians themselves. In the 1980s, scientists developed ways of identifying microbes by searching for their DNA amid environmental samples. This is now the main way of finding new species, but it means that many microbes —  like Loki and its kin —  are only known through their DNA. No one knows what they look like. “It’s high on our priority list,” says Ettema. “It’s crucial that we have a look at the cells to see what they’re doing, but that’s extremely hard. Loki was discovered in   ocean sediments, where you don’t go every day. Thor and Heimdall are present in shallower environments, but they’re less than 0. 1 percent of the total microbial community. It’s like looking for a needle in a haystack, but we’re working on it.']))

[2]


In [3]:
df3 = pd.read_pickle('cleaned_articles1.pkl')

In [4]:
df3.head()

,Unnamed: 0,id,title,publication,author,date,year,month,url,content,cleaned
0,0,17283,House Republicans Fret About Winning Their Hea...,New York Times,Carl Hulse,2016-12-31,2016.0,12.0,NaN,WASHINGTON — Congressional Republicans have...,washington congression republican new fear com...
1,1,17284,Rift Between Officers and Residents as Killing...,New York Times,Benjamin Mueller and Al Baker,2017-06-19,2017.0,6.0,NaN,"After the bullet shells get counted, the blood...",after bullet shell get count blood dri votiv c...
2,2,17285,"Tyrus Wong, ‘Bambi’ Artist Thwarted by Racial ...",New York Times,Margalit Fox,2017-01-06,2017.0,1.0,NaN,"When Walt Disney’s “Bambi” opened in 1942, cri...",when walt disney bambi open critic prais spare...
3,3,17286,"Among Deaths in 2016, a Heavy Toll in Pop Musi...",New York Times,William McDonald,2017-04-10,2017.0,4.0,NaN,"Death may be the great equalizer, but it isn’t...",death may great equal necessarili evenhand of ...
4,4,17287,Kim Jong-un Says North Korea Is Preparing to T...,New York Times,Choe Sang-Hun,2017-01-02,2017.0,1.0,NaN,"SEOUL, South Korea — North Korea’s leader, ...",seoul south korea north korea leader kim said ...
